In [1]:
import tensorflow as tf
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
# Minimum probability to filter weak detections
confidence_threshold = 0.5 

# Threshold when applyong non-maxima suppression
nms_threshold = 0.3 

# Load the COCO class labels the YOLO model was trained on
LABELS = open('model/coco.names').read().strip().split("\n")

# Initialize a list of colors to represent each possible class label
np.random.seed(42)
COLORS = np.random.randint(0, 255, size=(len(LABELS), 3), dtype="uint8")

# Load our YOLO object detector trained on the COCO dataset (80 classes)
net = cv.dnn.readNetFromDarknet('model/yolov3.cfg', 'model/yolov3.weights')

# Determine only the *output* layer names that we need from YOLO
ln = net.getLayerNames()
ln = [ln[i - 1] for i in net.getUnconnectedOutLayers()]
W = None
H = None

# Initialize the video stream


vs = cv.VideoCapture('pic3.jpg')

# Loop over frames from the video file stream
while vs.isOpened():
	# Read the next frame from the file
	(grabbed, frame) = vs.read()

	# If grabbed=False, we have reached the end of the stream
	if not grabbed:
		break

	if W is None or H is None:
		(H, W) = frame.shape[:2]
	blob = cv.dnn.blobFromImage(frame, 1 / 255.0, (416, 416), swapRB=True, crop=False)
	net.setInput(blob)
	layerOutputs = net.forward(ln)
	boxes = []
	confidences = []
	classIDs = []

	for output in layerOutputs:
		for detection in output:
			scores = detection[5:]
			classID = 0 #np.argmax(scores)
			confidence = scores[classID]

			if confidence > confidence_threshold:
				
				box = detection[0:4] * np.array([W, H, W, H])
				(centerX, centerY, width, height) = box.astype("int")

				x = int(centerX - (width / 2))
				y = int(centerY - (height / 2))
    
				boxes.append([x, y, int(width), int(height)])
				confidences.append(float(confidence))
				classIDs.append(classID)
				count=0
	idxs = cv.dnn.NMSBoxes(boxes, confidences, confidence_threshold, nms_threshold)
 
	count = 0
	if len(idxs) > 0:
		for i in idxs.flatten():
			# Extract the bounding box coordinates
			count += 1
			(x, y) = (boxes[i][0], boxes[i][1])
			(w, h) = (boxes[i][2], boxes[i][3])

			# Draw a bounding box rectangle and label on the frame
			color = (0,255, 0) #[int(c) for c in COLORS[classIDs[i]]]
			cv.rectangle(frame, (x, y), (x + w, y + h), color, 2)
			# text = "{}: {:.2f}".format(LABELS[classIDs[i]], confidences[i])
			text = "the picture has: {} persons".format(count)
			# cv.putText(frame, text, (x, y - 5), cv.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
			cv.putText(frame, text, (10,20), cv.FONT_HERSHEY_COMPLEX_SMALL, 0.5, color, 2)
   

	# show the output frame
	cv.imshow("Frame", frame)
	key = cv.waitKey(1) & 0xFF
	print(f"the picture has: {count} persons")
	# if the `q` key was pressed, break from the loop
	if key == ord("q"):
		break

vs.release()
# cv.destroyAllWindows()

the picture has: 5 persons
